In [1]:
import duckdb
import polars as pl
from rich import print
from IPython.display import display
%load_ext rich
con = duckdb.connect()

For the sake of simplicty, we clean out certain aspects of the data before:

In [2]:
df = pl.read_csv('data/all_seasons.csv')
df = df.unique(subset= ["player_name","season"])
qr = """
SELECT
    player_name,
    age,
    team_abbreviation,
    college,
    player_height,
    player_weight,
    draft_year,
    draft_round,
    draft_number,
    season
    FROM df,
    LIMIT 5;
"""
con.sql(qr)


┌──────────────────┬────────┬───────────────────┬────────────────┬───────────────┬───────────────┬────────────┬─────────────┬──────────────┬─────────┐
│   player_name    │  age   │ team_abbreviation │    college     │ player_height │ player_weight │ draft_year │ draft_round │ draft_number │ season  │
│     varchar      │ double │      varchar      │    varchar     │    double     │    double     │  varchar   │   varchar   │   varchar    │ varchar │
├──────────────────┼────────┼───────────────────┼────────────────┼───────────────┼───────────────┼────────────┼─────────────┼──────────────┼─────────┤
│ Cory Alexander   │   25.0 │ DEN               │ Virginia       │        185.42 │      86.18248 │ 1995       │ 1           │ 29           │ 1997-98 │
│ Yogi Ferrell     │   26.0 │ SAC               │ Indiana        │        182.88 │      81.64656 │ Undrafted  │ Undrafted   │ Undrafted    │ 2018-19 │
│ Jonas Jerebko    │   27.0 │ DET               │ None           │        208.28 │    104.779

In [3]:
qr = """
DROP TABLE IF EXISTS players_seasons;

DROP TYPE IF EXISTS season_stats;
CREATE TYPE season_stats AS STRUCT(
    season INTEGER,
    gp INTEGER,
    pts FLOAT,
    reb FLOAT,
    ast FLOAT
);

CREATE TABLE players (
    player_name TEXT,
    age INTEGER,
    team_abbreviation TEXT,
    college TEXT,
    player_height FLOAT,
    player_weight FLOAT,
    draft_year TEXT,
    draft_round TEXT,
    draft_number TEXT,
    season_stats season_stats[],
    current_season INTEGER,
    PRIMARY KEY(player_name, current_season, team_abbreviation, college)
)
"""
con.sql(qr)

In [4]:
qr="""
SELECT
    MIN(season[:4]::INTEGER) AS min_year,
    MAX(season[:4]::INTEGER) AS max_year,
    MIN(season) AS min_year_text,
    MAX(season) AS max_year_text,
FROM read_csv('data/all_seasons.csv')
"""
dfr = con.sql(qr).pl()
print(dfr)


shape: (1, 4)
┌──────────┬──────────┬───────────────┬───────────────┐
│ min_year ┆ max_year ┆ min_year_text ┆ max_year_text │
│ ---      ┆ ---      ┆ ---           ┆ ---           │
│ i32      ┆ i32      ┆ str           ┆ str           │
╞══════════╪══════════╪═══════════════╪═══════════════╡
│ 1996     ┆ 2022     ┆ 1996-97       ┆ 2022-23       │
└──────────┴──────────┴───────────────┴───────────────┘

In [5]:
qr = """
FROM players
LIMIT 5
"""
con.sql(qr)


┌─────────────┬───────┬───────────────────┬─────────┬───────────────┬───────────────┬────────────┬─────────────┬──────────────┬───────────────────────────────────────────────────────────────────────┬────────────────┐
│ player_name │  age  │ team_abbreviation │ college │ player_height │ player_weight │ draft_year │ draft_round │ draft_number │                             season_stats                              │ current_season │
│   varchar   │ int32 │      varchar      │ varchar │     float     │     float     │  varchar   │   varchar   │   varchar    │ struct(season integer, gp integer, pts float, reb float, ast float)[] │     int32      │
├─────────────┴───────┴───────────────────┴─────────┴───────────────┴───────────────┴────────────┴─────────────┴──────────────┴───────────────────────────────────────────────────────────────────────┴────────────────┤
│                                                                                                        0 rows                    

In [6]:
qr="""
WITH today AS (
    SELECT * FROM df
    WHERE season='1996-97'
)
FROM today
LIMIT 5
"""
con.sql(qr)


┌───────┬────────────────┬───────────────────┬────────┬───────────────┬────────────────────┬────────────┬─────────┬────────────┬─────────────┬──────────────┬───────┬────────┬────────┬────────┬────────────┬─────────────────────┬─────────────────────┬─────────────────────┬─────────────────────┬──────────────────────┬─────────┐
│  v0   │  player_name   │ team_abbreviation │  age   │ player_height │   player_weight    │  college   │ country │ draft_year │ draft_round │ draft_number │  gp   │  pts   │  reb   │  ast   │ net_rating │      oreb_pct       │      dreb_pct       │       usg_pct       │       ts_pct        │       ast_pct        │ season  │
│ int64 │    varchar     │      varchar      │ double │    double     │       double       │  varchar   │ varchar │  varchar   │   varchar   │   varchar    │ int64 │ double │ double │ double │   double   │       double        │       double        │       double        │       double        │        double        │ varchar │
├───────┼─────────

In [7]:
qr = """
WITH yesterday AS (
    SELECT * FROM players
    WHERE current_season={current_season}
), today AS (
    SELECT * FROM df
    WHERE season='{season}'
)
SELECT
    COALESCE(t.player_name, y.player_name) AS player_name,
    COALESCE(t.age, y.age) AS age,
    COALESCE(t.team_abbreviation, y.team_abbreviation) AS team_abbreviation,
    COALESCE(t.college, y.college) AS college,
    COALESCE(t.player_height, y.player_height) AS player_height,
    COALESCE(t.player_weight, y.player_weight) AS player_weight,
    COALESCE(t.draft_year, y.draft_year) AS draft_year,
    COALESCE(t.draft_round, y.draft_round) AS draft_round,
    COALESCE(t.draft_number, y.draft_number) AS draft_number,
    -- Update for season_stats
    CASE WHEN y.season_stats IS NULL
        THEN ARRAY[ROW(
            t.season[:4]::INTEGER,
            t.gp,
            t.pts,
            t.reb,
            t.ast
        )::season_stats]
    WHEN t.season IS NOT NULL
        THEN y.season_stats || ARRAY[ROW(
            t.season[:4]::INTEGER,
            t.gp,
            t.pts,
            t.reb,
            t.ast
        )::season_stats]
    ELSE y.season_stats
    END AS season_stats,
    -- Update for current_season,
    COALESCE(t.season[:4]::INTEGER, y.current_season+1) AS current_season
FROM today t
FULL OUTER JOIN yesterday y
ON t.player_name = y.player_name;
"""
query = qr.format(current_season=1995, season='1996-97')
print(query)
dfr = con.sql(query).pl()
display(dfr.head())

WITH yesterday AS (
    SELECT * FROM players
    WHERE current_season=1995
), today AS (
    SELECT * FROM df
    WHERE season='1996-97'
)
SELECT
    COALESCE(t.player_name, y.player_name) AS player_name,
    COALESCE(t.age, y.age) AS age,
    COALESCE(t.team_abbreviation, y.team_abbreviation) AS team_abbreviation,
    COALESCE(t.college, y.college) AS college,
    COALESCE(t.player_height, y.player_height) AS player_height,
    COALESCE(t.player_weight, y.player_weight) AS player_weight,
    COALESCE(t.draft_year, y.draft_year) AS draft_year,
    COALESCE(t.draft_round, y.draft_round) AS draft_round,
    COALESCE(t.draft_number, y.draft_number) AS draft_number,
    -- Update for season_stats
    CASE WHEN y.season_stats IS NULL
        THEN ARRAY[ROW(
            t.season[:4]::INTEGER,
            t.gp,
            t.pts,
            t.reb,
            t.ast
        )::season_stats]
    WHEN t.season IS NOT NULL
        THEN y.season_stats || ARRAY[ROW(
            t.season[:4]::INTEGER,
            t.gp,
            t.pts,
            t.reb,
            t.ast
        )::season_stats]
    ELSE y.season_stats
    END AS season_stats,
    -- Update for current_season,
    COALESCE(t.season[:4]::INTEGER, y.current_season+1) AS current_season
FROM today t
FULL OUTER JOIN yesterday y
ON t.player_name = y.player_name;

player_name,age,team_abbreviation,college,player_height,player_weight,draft_year,draft_round,draft_number,season_stats,current_season
str,f64,str,str,f64,f64,str,str,str,list[struct[5]],i32
"""Eric Williams""",24.0,"""BOS""","""Providence""",203.2,99.79024,"""1995""","""1""","""14""","[{1996,72,15.0,4.6,1.8}]",1996
"""Darrin Hancock""",25.0,"""ATL""","""Kansas""",200.66,92.98636,"""1994""","""2""","""38""","[{1996,24,1.8,0.8,0.5}]",1996
"""Vincent Askew""",31.0,"""DEN""","""Memphis""",198.12,106.59412,"""1987""","""2""","""39""","[{1996,43,5.6,2.3,2.1}]",1996
"""Eddie Johnson""",38.0,"""HOU""","""Illinois""",200.66,97.52228,"""1981""","""2""","""29""","[{1996,52,8.2,2.7,1.0}]",1996
"""Jim McIlvaine""",24.0,"""SEA""","""Marquette""",215.9,117.93392,"""1994""","""2""","""32""","[{1996,82,3.8,4.0,0.3}]",1996


In [8]:
query_insert = """INSERT INTO players""" + query
con.sql(query_insert)

In [9]:
for year in range(1997,2022):
    season = '-'.join([str(year+1),str(year+2)[2:]])
    query = qr.format(current_season=year, season=season)
    # print(query)
    query_insert = """INSERT INTO players""" + query
    # print(f"Inserted: {season}")
    con.sql(query_insert)

In [10]:
qr = """
SELECT * FROM players
WHERE current_season=2021
AND player_name='Kobe Bryant'
"""
con.sql(qr)



┌─────────────┬───────┬───────────────────┬─────────┬───────────────┬───────────────┬────────────┬─────────────┬──────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

In [11]:
qr = """
SELECT
    player_name,
    UNNEST(season_stats) FROM players
WHERE current_season=2021
AND player_name='Kobe Bryant'
"""
con.sql(qr)


┌─────────────┬─────────────────────────────────────────────────────────────────────┐
│ player_name │                        unnest(season_stats)                         │
│   varchar   │ struct(season integer, gp integer, pts float, reb float, ast float) │
├─────────────┼─────────────────────────────────────────────────────────────────────┤
│ Kobe Bryant │ {'season': 1998, 'gp': 50, 'pts': 19.9, 'reb': 5.3, 'ast': 3.8}     │
│ Kobe Bryant │ {'season': 1999, 'gp': 66, 'pts': 22.5, 'reb': 6.3, 'ast': 4.9}     │
│ Kobe Bryant │ {'season': 2000, 'gp': 68, 'pts': 28.5, 'reb': 5.9, 'ast': 5.0}     │
│ Kobe Bryant │ {'season': 2001, 'gp': 80, 'pts': 25.2, 'reb': 5.5, 'ast': 5.5}     │
│ Kobe Bryant │ {'season': 2002, 'gp': 82, 'pts': 30.0, 'reb': 6.9, 'ast': 5.9}     │
│ Kobe Bryant │ {'season': 2003, 'gp': 65, 'pts': 24.0, 'reb': 5.5, 'ast': 5.1}     │
│ Kobe Bryant │ {'season': 2004, 'gp': 66, 'pts': 27.6, 'reb': 5.9, 'ast': 6.0}     │
│ Kobe Bryant │ {'season': 2005, 'gp': 80, 'pts': 35.

In [12]:

qr = """
WITH expansion AS (
    SELECT
        player_name,
        UNNEST(season_stats)::season_stats AS season_stats
    FROM players
    WHERE current_season=2021
    AND player_name='Kobe Bryant'
)
SELECT
    player_name,
    season_stats.*
FROM expansion
"""
con.sql(qr)


┌─────────────┬────────┬───────┬───────┬───────┬───────┐
│ player_name │ season │  gp   │  pts  │  reb  │  ast  │
│   varchar   │ int32  │ int32 │ float │ float │ float │
├─────────────┼────────┼───────┼───────┼───────┼───────┤
│ Kobe Bryant │   1998 │    50 │  19.9 │   5.3 │   3.8 │
│ Kobe Bryant │   1999 │    66 │  22.5 │   6.3 │   4.9 │
│ Kobe Bryant │   2000 │    68 │  28.5 │   5.9 │   5.0 │
│ Kobe Bryant │   2001 │    80 │  25.2 │   5.5 │   5.5 │
│ Kobe Bryant │   2002 │    82 │  30.0 │   6.9 │   5.9 │
│ Kobe Bryant │   2003 │    65 │  24.0 │   5.5 │   5.1 │
│ Kobe Bryant │   2004 │    66 │  27.6 │   5.9 │   6.0 │
│ Kobe Bryant │   2005 │    80 │  35.4 │   5.3 │   4.5 │
│ Kobe Bryant │   2006 │    77 │  31.6 │   5.7 │   5.4 │
│ Kobe Bryant │   2007 │    82 │  28.3 │   6.3 │   5.4 │
│ Kobe Bryant │   2008 │    82 │  26.8 │   5.2 │   4.9 │
│ Kobe Bryant │   2009 │    73 │  27.0 │   5.4 │   5.0 │
│ Kobe Bryant │   2010 │    82 │  25.3 │   5.1 │   4.7 │
│ Kobe Bryant │   2011 │    58

In [13]:
qr = """
DROP TYPE IF EXISTS scoring_class;
CREATE TYPE scoring_class AS ENUM ('star', 'good', 'average', 'bad');
"""
con.sql(qr)
qr = """
DROP TABLE IF EXISTS players;
CREATE TABLE players (
    player_name TEXT,
    age INTEGER,
    team_abbreviation TEXT,
    college TEXT,
    player_height FLOAT,
    player_weight FLOAT,
    draft_year TEXT,
    draft_round TEXT,
    draft_number TEXT,
    season_stats season_stats[],
    scoring_class scoring_class,
    years_since_last_season INTEGER,
    current_season INTEGER,
    PRIMARY KEY(player_name, current_season, team_abbreviation, college)
)
"""
con.sql(qr)

In [14]:
qr = """
WITH yesterday AS (
    SELECT * FROM players
    WHERE current_season={current_season}
), today AS (
    SELECT * FROM df
    WHERE season='{season}'
)
SELECT
    COALESCE(t.player_name, y.player_name) AS player_name,
    COALESCE(t.age, y.age) AS age,
    COALESCE(t.team_abbreviation, y.team_abbreviation) AS team_abbreviation,
    COALESCE(t.college, y.college) AS college,
    COALESCE(t.player_height, y.player_height) AS player_height,
    COALESCE(t.player_weight, y.player_weight) AS player_weight,
    COALESCE(t.draft_year, y.draft_year) AS draft_year,
    COALESCE(t.draft_round, y.draft_round) AS draft_round,
    COALESCE(t.draft_number, y.draft_number) AS draft_number,
    -- Update for season_stats
    CASE WHEN y.season_stats IS NULL
        THEN ARRAY[ROW(
            t.season[:4]::INTEGER,
            t.gp,
            t.pts,
            t.reb,
            t.ast
        )::season_stats]
    WHEN t.season IS NOT NULL
        THEN y.season_stats || ARRAY[ROW(
            t.season[:4]::INTEGER,
            t.gp,
            t.pts,
            t.reb,
            t.ast
        )::season_stats]
    ELSE y.season_stats
    END AS season_stats,
    -- Scoring
    CASE
        WHEN t.season IS NOT NULL THEN
            CASE
                WHEN t.pts > 20 THEN 'star'
                WHEN t.pts > 15 THEN 'good'
                WHEN t.pts > 10 THEN 'average'
                ELSE 'bad'
            END::scoring_class
        ELSE y.scoring_class
    END AS scoring_class,
    -- Years active
    CASE WHEN t.season IS NOT NULL THEN 0
        ELSE y.years_since_last_season + 1
    END AS years_since_last_season,
    -- Update for current_season,
    COALESCE(t.season[:4]::INTEGER, y.current_season+1) AS current_season
FROM today t
FULL OUTER JOIN yesterday y
ON t.player_name = y.player_name;
"""
query = qr.format(current_season=1995, season='1996-97')
print(query)
dfr = con.sql(query).pl()
display(dfr.head())

WITH yesterday AS (
    SELECT * FROM players
    WHERE current_season=1995
), today AS (
    SELECT * FROM df
    WHERE season='1996-97'
)
SELECT
    COALESCE(t.player_name, y.player_name) AS player_name,
    COALESCE(t.age, y.age) AS age,
    COALESCE(t.team_abbreviation, y.team_abbreviation) AS team_abbreviation,
    COALESCE(t.college, y.college) AS college,
    COALESCE(t.player_height, y.player_height) AS player_height,
    COALESCE(t.player_weight, y.player_weight) AS player_weight,
    COALESCE(t.draft_year, y.draft_year) AS draft_year,
    COALESCE(t.draft_round, y.draft_round) AS draft_round,
    COALESCE(t.draft_number, y.draft_number) AS draft_number,
    -- Update for season_stats
    CASE WHEN y.season_stats IS NULL
        THEN ARRAY[ROW(
            t.season[:4]::INTEGER,
            t.gp,
            t.pts,
            t.reb,
            t.ast
        )::season_stats]
    WHEN t.season IS NOT NULL
        THEN y.season_stats || ARRAY[ROW(
            t.season[:4]::INTEGER,
            t.gp,
            t.pts,
            t.reb,
            t.ast
        )::season_stats]
    ELSE y.season_stats
    END AS season_stats,
    -- Scoring
    CASE
        WHEN t.season IS NOT NULL THEN
            CASE
                WHEN t.pts > 20 THEN 'star'
                WHEN t.pts > 15 THEN 'good'
                WHEN t.pts > 10 THEN 'average'
                ELSE 'bad'
            END::scoring_class
        ELSE y.scoring_class
    END AS scoring_class,
    -- Years active
    CASE WHEN t.season IS NOT NULL THEN 0
        ELSE y.years_since_last_season + 1
    END AS years_since_last_season,
    -- Update for current_season,
    COALESCE(t.season[:4]::INTEGER, y.current_season+1) AS current_season
FROM today t
FULL OUTER JOIN yesterday y
ON t.player_name = y.player_name;

player_name,age,team_abbreviation,college,player_height,player_weight,draft_year,draft_round,draft_number,season_stats,scoring_class,years_since_last_season,current_season
str,f64,str,str,f64,f64,str,str,str,list[struct[5]],cat,i32,i32
"""Eric Williams""",24.0,"""BOS""","""Providence""",203.2,99.79024,"""1995""","""1""","""14""","[{1996,72,15.0,4.6,1.8}]","""average""",0,1996
"""Darrin Hancock""",25.0,"""ATL""","""Kansas""",200.66,92.98636,"""1994""","""2""","""38""","[{1996,24,1.8,0.8,0.5}]","""bad""",0,1996
"""Vincent Askew""",31.0,"""DEN""","""Memphis""",198.12,106.59412,"""1987""","""2""","""39""","[{1996,43,5.6,2.3,2.1}]","""bad""",0,1996
"""Eddie Johnson""",38.0,"""HOU""","""Illinois""",200.66,97.52228,"""1981""","""2""","""29""","[{1996,52,8.2,2.7,1.0}]","""bad""",0,1996
"""Jim McIlvaine""",24.0,"""SEA""","""Marquette""",215.9,117.93392,"""1994""","""2""","""32""","[{1996,82,3.8,4.0,0.3}]","""bad""",0,1996


In [15]:
query = qr.format(current_season=1996, season='1997-98')
print(query)
dfr = con.sql(query).pl()
display(dfr.head())

WITH yesterday AS (
    SELECT * FROM players
    WHERE current_season=1996
), today AS (
    SELECT * FROM df
    WHERE season='1997-98'
)
SELECT
    COALESCE(t.player_name, y.player_name) AS player_name,
    COALESCE(t.age, y.age) AS age,
    COALESCE(t.team_abbreviation, y.team_abbreviation) AS team_abbreviation,
    COALESCE(t.college, y.college) AS college,
    COALESCE(t.player_height, y.player_height) AS player_height,
    COALESCE(t.player_weight, y.player_weight) AS player_weight,
    COALESCE(t.draft_year, y.draft_year) AS draft_year,
    COALESCE(t.draft_round, y.draft_round) AS draft_round,
    COALESCE(t.draft_number, y.draft_number) AS draft_number,
    -- Update for season_stats
    CASE WHEN y.season_stats IS NULL
        THEN ARRAY[ROW(
            t.season[:4]::INTEGER,
            t.gp,
            t.pts,
            t.reb,
            t.ast
        )::season_stats]
    WHEN t.season IS NOT NULL
        THEN y.season_stats || ARRAY[ROW(
            t.season[:4]::INTEGER,
            t.gp,
            t.pts,
            t.reb,
            t.ast
        )::season_stats]
    ELSE y.season_stats
    END AS season_stats,
    -- Scoring
    CASE
        WHEN t.season IS NOT NULL THEN
            CASE
                WHEN t.pts > 20 THEN 'star'
                WHEN t.pts > 15 THEN 'good'
                WHEN t.pts > 10 THEN 'average'
                ELSE 'bad'
            END::scoring_class
        ELSE y.scoring_class
    END AS scoring_class,
    -- Years active
    CASE WHEN t.season IS NOT NULL THEN 0
        ELSE y.years_since_last_season + 1
    END AS years_since_last_season,
    -- Update for current_season,
    COALESCE(t.season[:4]::INTEGER, y.current_season+1) AS current_season
FROM today t
FULL OUTER JOIN yesterday y
ON t.player_name = y.player_name;

player_name,age,team_abbreviation,college,player_height,player_weight,draft_year,draft_round,draft_number,season_stats,scoring_class,years_since_last_season,current_season
str,f64,str,str,f64,f64,str,str,str,list[struct[5]],cat,i32,i32
"""Cory Alexander""",25.0,"""DEN""","""Virginia""",185.42,86.18248,"""1995""","""1""","""29""","[{1997,60,8.1,2.4,3.5}]","""bad""",0,1997
"""John Crotty""",28.0,"""POR""","""Virginia""",185.42,83.91452,"""Undrafted""","""Undrafted""","""Undrafted""","[{1997,26,3.7,1.2,2.4}]","""bad""",0,1997
"""Steve Henson""",30.0,"""DET""","""Kansas State""",185.42,81.64656,"""1990""","""2""","""44""","[{1997,23,1.6,0.1,0.2}]","""bad""",0,1997
"""Anthony Bowie""",34.0,"""NYK""","""Oklahoma""",198.12,90.7184,"""1986""","""3""","""66""","[{1997,27,2.8,1.0,0.4}]","""bad""",0,1997
"""Mahmoud Abdul-Rauf""",29.0,"""SAC""","""Louisiana State""",185.42,73.481904,"""1990""","""1""","""3""","[{1997,31,7.3,1.2,1.9}]","""bad""",0,1997


In [16]:
for year in range(1998,2022):
    season = '-'.join([str(year+1),str(year+2)[2:]])
    query = qr.format(current_season=year, season=season)
    # print(query)
    query_insert = """INSERT INTO players""" + query
    # print(f"Inserted: {season}")
    con.sql(query_insert)

In [17]:
qr="""
SELECT COUNT(*), player_name FROM players
GROUP BY player_name
HAVING COUNT(*)>1
LIMIT 5;
"""
print(
    con.sql(qr).pl()
)

shape: (5, 2)
┌──────────────┬─────────────────┐
│ count_star() ┆ player_name     │
│ ---          ┆ ---             │
│ i64          ┆ str             │
╞══════════════╪═════════════════╡
│ 24           ┆ Jeff McInnis    │
│ 24           ┆ Arvydas Sabonis │
│ 24           ┆ Detlef Schrempf │
│ 24           ┆ Peja Stojakovic │
│ 24           ┆ Andrew DeClercq │
└──────────────┴─────────────────┘

In [18]:
qr = """
SELECT * FROM players
WHERE player_name='Michael Curry'
AND current_season=2022
"""
dfr=con.sql(qr).pl()
print(dfr)

shape: (1, 13)
┌────────────┬─────┬────────────┬────────────┬───┬────────────┬────────────┬───────────┬───────────┐
│ player_nam ┆ age ┆ team_abbre ┆ college    ┆ … ┆ season_sta ┆ scoring_cl ┆ years_sin ┆ current_s │
│ e          ┆ --- ┆ viation    ┆ ---        ┆   ┆ ts         ┆ ass        ┆ ce_last_s ┆ eason     │
│ ---        ┆ i32 ┆ ---        ┆ str        ┆   ┆ ---        ┆ ---        ┆ eason     ┆ ---       │
│ str        ┆     ┆ str        ┆            ┆   ┆ list[struc ┆ cat        ┆ ---       ┆ i32       │
│            ┆     ┆            ┆            ┆   ┆ t[5]]      ┆            ┆ i32       ┆           │
╞════════════╪═════╪════════════╪════════════╪═══╪════════════╪════════════╪═══════════╪═══════════╡
│ Michael    ┆ 36  ┆ IND        ┆ Georgia    ┆ … ┆ [{1999,82, ┆ bad        ┆ 18        ┆ 2022      │
│ Curry      ┆     ┆            ┆ Southern   ┆   ┆ 6.2,1.3,1. ┆            ┆           ┆           │
│            ┆     ┆            ┆            ┆   ┆ 1},        ┆            ┆           ┆           │
│            ┆     ┆            ┆            ┆   ┆ {2000,…    ┆            ┆           ┆           │
└────────────┴─────┴────────────┴────────────┴───┴────────────┴────────────┴───────────┴───────────┘

In [19]:
qr = """
SELECT
    player_name,
    array_length(season_stats) AS total_seasons,
    (season_stats[1]::season_stats).pts AS first_season,
    (season_stats[array_length(season_stats)]::season_stats).pts AS last_season,
FROM players
WHERE player_name='Michael Curry'
AND current_season=2022
"""
dfr=con.sql(qr).pl()
print(dfr)

shape: (1, 4)
┌───────────────┬───────────────┬──────────────┬─────────────┐
│ player_name   ┆ total_seasons ┆ first_season ┆ last_season │
│ ---           ┆ ---           ┆ ---          ┆ ---         │
│ str           ┆ i64           ┆ f32          ┆ f32         │
╞═══════════════╪═══════════════╪══════════════╪═════════════╡
│ Michael Curry ┆ 6             ┆ 6.2          ┆ 1.7         │
└───────────────┴───────────────┴──────────────┴─────────────┘

In [20]:
qr = """
WITH points_fl AS (
SELECT
    player_name,
    array_length(season_stats) AS total_seasons,
    (season_stats[1]::season_stats).pts AS first_season,
    (season_stats[array_length(season_stats)]::season_stats).pts AS last_season,
    current_season
FROM players
)
SELECT
    player_name,
    total_seasons,
    last_season /
        CASE
            WHEN first_season=0 THEN 1 ELSE first_season
        END
    AS ratio,
    current_season
FROM points_fl
WHERE current_season=2022
ORDER BY ratio DESC
LIMIT 10
"""
dfr=con.sql(qr).pl()
print(dfr)

shape: (10, 4)
┌──────────────────┬───────────────┬───────────┬────────────────┐
│ player_name      ┆ total_seasons ┆ ratio     ┆ current_season │
│ ---              ┆ ---           ┆ ---       ┆ ---            │
│ str              ┆ i64           ┆ f32       ┆ i32            │
╞══════════════════╪═══════════════╪═══════════╪════════════════╡
│ Brandon Williams ┆ 2             ┆ 42.999996 ┆ 2022           │
│ David Wingate    ┆ 2             ┆ 20.0      ┆ 2022           │
│ Ndudi Ebi        ┆ 2             ┆ 16.875    ┆ 2022           │
│ Pavel Podkolzin  ┆ 2             ┆ 15.0      ┆ 2022           │
│ Julius Randle    ┆ 9             ┆ 12.55     ┆ 2022           │
│ Terry Rozier     ┆ 8             ┆ 11.722222 ┆ 2022           │
│ Nathan Jawai     ┆ 2             ┆ 10.666666 ┆ 2022           │
│ Jared Harper     ┆ 3             ┆ 10.571429 ┆ 2022           │
│ Nick Richards    ┆ 3             ┆ 10.25     ┆ 2022           │
│ Louis King       ┆ 4             ┆ 10.0      ┆ 2022           │
└──────────────────┴───────────────┴───────────┴────────────────┘